# Advanced Technical Features for Stock Direction Prediction

## Goal
Improve directional accuracy from 58.81% to >60% by adding:
1. **15 advanced technical features** (market context, momentum, volume, statistical)
2. **XGBoost ensemble** with Linear model (70-30 weighted)
3. **Class weighting** for balanced predictions

## Baseline
- Model: Linear Regression + Logistic Regression
- Accuracy: 58.81%
- Sharpe: 0.2095
- Features: 60-day window × 32 technical indicators

## Target
- Accuracy: >60%
- Maintain Sharpe ≥0.21

---
## 1. Imports and Setup

In [1]:
# Standard libraries
import os
import glob
import warnings
import time
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# Progress bars
from tqdm.auto import tqdm

# MLflow
import mlflow
import mlflow.sklearn

# API requests (for NIFTY 50 fetching)
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds
np.random.seed(42)

print("✅ All libraries imported successfully")
print(f"XGBoost version: {xgb.__version__}")
print(f"Pandas version: {pd.__version__}")

✅ All libraries imported successfully
XGBoost version: 3.1.1
Pandas version: 2.3.3


In [2]:
# Configuration
WINDOW_SIZE = 60          # Input: past 60 days of features
HORIZON = 7               # Output: predict 7-day ahead return & direction
TEST_STOCK_RATIO = 0.2    # 20% of stocks for testing
RANDOM_STATE = 42

# Paths
DATA_DIR = Path("data/processed/stock_data")
MODELS_DIR = Path("models")
MLFLOW_DIR = Path("mlruns")

MODELS_DIR.mkdir(exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"MLflow tracking: {MLFLOW_DIR}")

Data directory: data/processed/stock_data
Models directory: models
MLflow tracking: mlruns


In [3]:
# MLflow Setup
mlflow.set_tracking_uri(f"file://{MLFLOW_DIR.absolute()}")
mlflow.set_experiment("stock-return-prediction-multihorizon")

experiment = mlflow.get_experiment_by_name("stock-return-prediction-multihorizon")
print(f"MLflow Experiment: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")

MLflow Experiment: stock-return-prediction-multihorizon
Experiment ID: 785296905155056667
Artifact Location: file:///home/labeeb/Desktop/stock-curator/mlruns/785296905155056667


---
## 2. Load & Preprocess Data

In [4]:
# Load all stock CSV files
csv_files = sorted(glob.glob(str(DATA_DIR / "*_historical.csv")))

print(f"Found {len(csv_files)} stock CSV files")
print(f"\nFirst 5 files:")
for f in csv_files[:5]:
    print(f"  - {Path(f).name}")

Found 98 stock CSV files

First 5 files:
  - ABBOTINDIA_historical.csv
  - ABB_historical.csv
  - ADANIENT_historical.csv
  - ADANIGREEN_historical.csv
  - ADANIPORTS_historical.csv


In [5]:
def load_stock_data(file_path):
    """
    Load a single stock CSV and preprocess it.
    """
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    df_clean = df.dropna()
    
    # Need at least WINDOW_SIZE + HORIZON rows
    if len(df_clean) < WINDOW_SIZE + HORIZON:
        return None
    
    return df_clean


# Load all stocks
stock_data = {}
skipped = []

for file_path in csv_files:
    symbol = Path(file_path).stem.replace('_historical', '')
    df = load_stock_data(file_path)
    
    if df is not None:
        stock_data[symbol] = df
    else:
        skipped.append(symbol)

print(f"Loaded {len(stock_data)} stocks successfully")
if skipped:
    print(f"Skipped {len(skipped)} stocks: {skipped}")

Loaded 98 stocks successfully


In [6]:
# Inspect sample data
sample_symbol = list(stock_data.keys())[0]
sample_df = stock_data[sample_symbol]

print(f"\nSample Stock: {sample_symbol}")
print(f"Shape: {sample_df.shape}")
print(f"Date Range: {sample_df['Date'].min()} to {sample_df['Date'].max()}")
print(f"\nColumns ({len(sample_df.columns)}):")
print(sample_df.columns.tolist())
print(f"\nFirst 3 rows:")
print(sample_df.head(3))


Sample Stock: ABBOTINDIA
Shape: (447, 28)
Date Range: 2024-01-15 00:00:00+05:30 to 2025-10-31 00:00:00+05:30

Columns (28):
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OI', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'Volume_SMA_20', 'Volume_Ratio', 'Daily_Return', 'Price_Range', 'Price_Change', 'Symbol', 'Name', 'ISIN']

First 3 rows:
                        Date     Open      High       Low     Close  Volume  \
49 2024-01-15 00:00:00+05:30  25700.0  26381.65  25700.00  26210.45   33616   
50 2024-01-16 00:00:00+05:30  26220.0  26305.90  25601.00  25806.50   51548   
51 2024-01-17 00:00:00+05:30  25600.0  26139.95  25546.15  25730.70   15925   

    OI     SMA_5     SMA_10      SMA_20  ...      BB_Upper      BB_Lower  \
49   0  25228.03  24418.650  23511.0825  ...  25865.034135  21157.130865   
50   0  25542.95  24666.575  23662.3925  ...  26200.405545  21124.379455   
51   0  25767

---
## 3. Feature Engineering - Add Return-Based Features

In [7]:
def add_return_features(df):
    """
    Add return-based features for better stationarity.
    """
    df = df.copy()
    
    # Historical returns (already have Daily_Return = 1-day return)
    df['Return_3d'] = df['Close'].pct_change(3) * 100
    df['Return_5d'] = df['Close'].pct_change(5) * 100
    df['Return_10d'] = df['Close'].pct_change(10) * 100
    
    # Log returns for better stationarity
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1)) * 100
    
    # Volatility (rolling std of returns)
    df['Volatility_5d'] = df['Daily_Return'].rolling(5).std()
    df['Volatility_20d'] = df['Daily_Return'].rolling(20).std()
    
    # Momentum indicators
    df['Momentum_10d'] = df['Close'] - df['Close'].shift(10)
    df['Momentum_20d'] = df['Close'] - df['Close'].shift(20)
    
    # Drop NaN rows created by new features
    df = df.dropna()
    
    return df


# Apply to all stocks
print("Adding return-based features...")
for symbol in stock_data.keys():
    stock_data[symbol] = add_return_features(stock_data[symbol])

print(f"Feature engineering complete")
print(f"\nUpdated feature count: {len([col for col in stock_data[sample_symbol].columns if col not in ['Date', 'Symbol', 'Name', 'ISIN']])}")

Adding return-based features...
Feature engineering complete

Updated feature count: 32


---
## 3.5 Fetch NIFTY 50 Index Data (for Market Context Features)

In [8]:
# NIFTY 50 configuration
NIFTY50_INSTRUMENT_KEY = "NSE_INDEX|Nifty 50"
UPSTOX_ACCESS_TOKEN = os.getenv("UPSTOX_ACCESS_TOKEN")

def create_session_with_retries(retries=3, backoff_factor=0.3):
    """Create a requests session with retry logic"""
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 504),
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

session = create_session_with_retries()
print("✅ HTTP session created with retry logic")

✅ HTTP session created with retry logic


In [9]:
def fetch_nifty50_data(start_date, end_date, access_token=None):
    """
    Fetch NIFTY 50 index historical data

    Args:
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        access_token: Upstox API token

    Returns:
        DataFrame with Date and Close columns
    """
    if access_token is None:
        access_token = UPSTOX_ACCESS_TOKEN

    if not access_token:
        print("⚠️  Error: UPSTOX_ACCESS_TOKEN not set")
        return None

    # Build URL for NIFTY 50
    instrument_key_encoded = "NSE_INDEX%7CNifty%2050"  # URL encoded
    url = f"https://api.upstox.com/v3/historical-candle/{instrument_key_encoded}/days/1/{end_date}/{start_date}"

    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    try:
        response = session.get(url, headers=headers, timeout=15)

        if response.status_code == 200:
            data = response.json()

            if data.get('status') == 'success' and 'data' in data:
                candles = data['data'].get('candles', [])

                if not candles:
                    print("⚠️  No NIFTY 50 data returned")
                    return None

                # Convert to DataFrame
                df = pd.DataFrame(candles, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OI'])
                df['Date'] = pd.to_datetime(df['Date'])
                df = df.sort_values('Date').reset_index(drop=True)

                # Keep only Date and Close for market context
                df = df[['Date', 'Close']]
                df.rename(columns={'Close': 'Nifty50_Close'}, inplace=True)

                return df
            else:
                print(f"⚠️  API error: {data.get('message', 'Unknown error')}")
                return None
        else:
            print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")
            return None

    except Exception as e:
        print(f"⚠️  Exception: {str(e)}")
        return None

print("✅ fetch_nifty50_data() function defined")

✅ fetch_nifty50_data() function defined


In [10]:
# Fetch NIFTY 50 data for the same date range as stocks
sample_stock = list(stock_data.values())[0]
start_date = sample_stock['Date'].min().strftime('%Y-%m-%d')
end_date = sample_stock['Date'].max().strftime('%Y-%m-%d')

print(f"Fetching NIFTY 50 data from {start_date} to {end_date}...")

nifty50_df = fetch_nifty50_data(start_date, end_date)

if nifty50_df is not None:
    print(f"✅ Fetched {len(nifty50_df)} days of NIFTY 50 data")
    print(f"   Date range: {nifty50_df['Date'].min()} to {nifty50_df['Date'].max()}")
    print(f"\nFirst 3 rows:")
    print(nifty50_df.head(3))
else:
    print("⚠️  Failed to fetch NIFTY 50 data")
    print("   Will skip market context features (3 features)")

Fetching NIFTY 50 data from 2024-02-13 to 2025-10-31...
✅ Fetched 427 days of NIFTY 50 data
   Date range: 2024-02-13 00:00:00+05:30 to 2025-10-31 00:00:00+05:30

First 3 rows:
                       Date  Nifty50_Close
0 2024-02-13 00:00:00+05:30       21743.25
1 2024-02-14 00:00:00+05:30       21840.05
2 2024-02-15 00:00:00+05:30       21910.75


---
## 3.6 Advanced Technical Feature Engineering

Adding 15 new features across 4 categories:
1. **Market Context (3)**: Relative strength, correlation, market regime
2. **Momentum & Mean Reversion (6)**: RSI divergence, MACD crossover, BB squeeze, etc.
3. **Volume & Liquidity (3)**: Volume price trend, on-balance volume, volume breakout
4. **Statistical (3)**: Returns skewness, kurtosis, Hurst exponent

In [11]:
def add_advanced_technical_features(df, nifty50_df=None):
    """
    Add 15 advanced technical features to stock DataFrame

    Args:
        df: Stock DataFrame with existing 32 features
        nifty50_df: NIFTY 50 DataFrame with Date and Nifty50_Close columns

    Returns:
        DataFrame with up to 47 features (32 + 15, or 32 + 12 if no NIFTY data)
    """
    df = df.copy()

    # ========== MARKET CONTEXT FEATURES (3) ==========
    if nifty50_df is not None:
        # Merge with NIFTY 50 data
        df = df.merge(nifty50_df, on='Date', how='left')

        # 1. Relative strength to NIFTY 50
        df['Nifty50_Return'] = df['Nifty50_Close'].pct_change() * 100
        df['relative_strength_to_nifty50'] = df['Daily_Return'] - df['Nifty50_Return']

        # 2. Correlation to NIFTY 50 (20-day rolling)
        df['correlation_to_nifty50_20d'] = df['Daily_Return'].rolling(20).corr(df['Nifty50_Return'])

        # 3. Market regime (based on NIFTY 50 trend)
        nifty_sma_20 = df['Nifty50_Close'].rolling(20).mean()
        nifty_sma_50 = df['Nifty50_Close'].rolling(50).mean()
        df['market_regime'] = 0  # Sideways
        df.loc[nifty_sma_20 > nifty_sma_50, 'market_regime'] = 1  # Bull
        df.loc[nifty_sma_20 < nifty_sma_50, 'market_regime'] = -1  # Bear

        # Drop temporary columns
        df.drop(columns=['Nifty50_Close', 'Nifty50_Return'], inplace=True)
    else:
        # Set to NaN if NIFTY data not available
        df['relative_strength_to_nifty50'] = np.nan
        df['correlation_to_nifty50_20d'] = np.nan
        df['market_regime'] = 0  # Default to sideways

    # ========== MOMENTUM & MEAN REVERSION FEATURES (6) ==========

    # 4. RSI divergence (RSI trend vs price trend)
    rsi_change = df['RSI'].diff(5)
    price_change = df['Close'].pct_change(5)
    df['rsi_divergence'] = np.sign(rsi_change) - np.sign(price_change)

    # 5. MACD crossover signal
    df['macd_crossover_signal'] = 0
    macd_cross_up = (df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))
    macd_cross_down = (df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))
    df.loc[macd_cross_up, 'macd_crossover_signal'] = 1
    df.loc[macd_cross_down, 'macd_crossover_signal'] = -1

    # 6. Bollinger Band squeeze (volatility compression)
    bb_width = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['bb_squeeze'] = bb_width.rolling(20).apply(lambda x: (x.iloc[-1] - x.min()) / (x.max() - x.min() + 1e-8), raw=False)

    # 7. Price vs SMA50 percentage
    df['price_vs_sma50_pct'] = ((df['Close'] - df['SMA_50']) / df['SMA_50']) * 100

    # 8. Momentum strength (rate of change in momentum)
    df['momentum_strength'] = df['Momentum_10d'].diff(5)

    # 9. Support/Resistance distance (distance to 20-day high/low)
    high_20 = df['High'].rolling(20).max()
    low_20 = df['Low'].rolling(20).min()
    df['support_resistance_distance'] = np.where(
        df['Close'] > df['Close'].shift(1),
        (high_20 - df['Close']) / df['Close'],  # Distance to resistance
        (df['Close'] - low_20) / df['Close']     # Distance to support
    )

    # ========== VOLUME & LIQUIDITY FEATURES (3) ==========

    # 10. Volume Price Trend (cumulative volume × price direction)
    price_direction = np.sign(df['Close'].diff())
    df['volume_price_trend'] = (df['Volume'] * price_direction).cumsum()

    # 11. On-Balance Volume (cumulative volume flow)
    obv = np.where(df['Close'] > df['Close'].shift(1), df['Volume'],
                   np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0))
    df['on_balance_volume'] = obv.cumsum()

    # 12. Volume breakout (binary: volume > 2× average)
    df['volume_breakout'] = (df['Volume'] > 2 * df['Volume_SMA_20']).astype(int)

    # ========== STATISTICAL FEATURES (3) ==========

    # 13. Returns skewness (20-day)
    df['returns_skewness_20d'] = df['Daily_Return'].rolling(20).skew()

    # 14. Returns kurtosis (20-day) - tail risk
    df['returns_kurtosis_20d'] = df['Daily_Return'].rolling(20).kurt()

    # 15. Hurst exponent (trend persistence over 60 days)
    def calculate_hurst(ts, lags=range(2, 20)):
        """Calculate Hurst exponent - measures trend persistence"""
        if len(ts) < max(lags):
            return 0.5  # Random walk

        tau = []
        lagvec = []

        for lag in lags:
            # Calculate standard deviation of lagged differences
            pp = np.subtract(ts[lag:], ts[:-lag])
            lagvec.append(lag)
            tau.append(np.std(pp))

        # Linear fit in log-log scale
        try:
            poly = np.polyfit(np.log(lagvec), np.log(tau), 1)
            return poly[0]  # Slope = Hurst exponent
        except:
            return 0.5

    df['hurst_exponent'] = df['Close'].rolling(60).apply(lambda x: calculate_hurst(x.values), raw=False)

    # Drop NaN rows created by new features
    df = df.dropna()

    return df

print("✅ add_advanced_technical_features() function defined")

✅ add_advanced_technical_features() function defined


In [12]:
# Apply advanced features to all stocks
print("Adding 15 advanced technical features to all stocks...")
print("This may take a few minutes...")

for symbol in tqdm(stock_data.keys(), desc="Processing stocks"):
    stock_data[symbol] = add_advanced_technical_features(
        stock_data[symbol],
        nifty50_df if nifty50_df is not None else None
    )

print("\n✅ Advanced feature engineering complete!")

# Check feature count
sample_symbol = list(stock_data.keys())[0]
exclude_cols = ['Date', 'Symbol', 'Name', 'ISIN']
feature_cols = [col for col in stock_data[sample_symbol].columns if col not in exclude_cols]

print(f"\n📊 Feature count: {len(feature_cols)} features")
print(f"   Previous: 32 features")
print(f"   New: {len(feature_cols) - 32} features added")

# Print new features
new_features = [col for col in feature_cols if col not in [
    'Open', 'High', 'Low', 'Close', 'Volume', 'OI', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50',
    'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'BB_Middle', 'BB_Upper',
    'BB_Lower', 'Volume_SMA_20', 'Volume_Ratio', 'Daily_Return', 'Price_Range', 'Price_Change',
    'Return_3d', 'Return_5d', 'Return_10d', 'Log_Return', 'Volatility_5d', 'Volatility_20d',
    'Momentum_10d', 'Momentum_20d'
]]

print(f"\n📝 New features added:")
for i, feat in enumerate(new_features, 1):
    print(f"   {i:2d}. {feat}")

Adding 15 advanced technical features to all stocks...
This may take a few minutes...


Processing stocks:   0%|          | 0/98 [00:00<?, ?it/s]


✅ Advanced feature engineering complete!

📊 Feature count: 47 features
   Previous: 32 features
   New: 15 features added

📝 New features added:
    1. relative_strength_to_nifty50
    2. correlation_to_nifty50_20d
    3. market_regime
    4. rsi_divergence
    5. macd_crossover_signal
    6. bb_squeeze
    7. price_vs_sma50_pct
    8. momentum_strength
    9. support_resistance_distance
   10. volume_price_trend
   11. on_balance_volume
   12. volume_breakout
   13. returns_skewness_20d
   14. returns_kurtosis_20d
   15. hurst_exponent


---
## 4. Create Target Variables: Returns & Direction

In [13]:
def create_sequences_with_returns(df, window_size=60, horizon=7):
    """
    Create sliding window sequences with return and direction targets.
    
    Returns:
        X: (n_samples, window_size, n_features) - input features
        y_return: (n_samples,) - 7-day cumulative return %
        y_direction: (n_samples,) - 1 if UP, 0 if DOWN
    """
    features = df[feature_cols].values
    close_prices = df['Close'].values
    
    X, y_return, y_direction = [], [], []
    
    for i in range(len(df) - window_size - horizon + 1):
        # Input: past window_size days of all features
        X.append(features[i:i + window_size])
        
        # Current close price (last day of input window)
        current_close = close_prices[i + window_size - 1]
        
        # Future close price (horizon days ahead)
        future_close = close_prices[i + window_size + horizon - 1]
        
        # Calculate 7-day return percentage
        return_pct = ((future_close - current_close) / current_close) * 100
        y_return.append(return_pct)
        
        # Calculate direction (1 = UP, 0 = DOWN)
        direction = 1 if future_close > current_close else 0
        y_direction.append(direction)
    
    return np.array(X), np.array(y_return), np.array(y_direction)


# Test on sample stock
X_test, y_ret_test, y_dir_test = create_sequences_with_returns(stock_data[sample_symbol], WINDOW_SIZE, HORIZON)

print(f"\nTest Sequence Creation on {sample_symbol}:")
print(f"  X shape:          {X_test.shape}  <- (samples, window_size, features)")
print(f"  y_return shape:   {y_ret_test.shape}  <- 7-day return %")
print(f"  y_direction shape: {y_dir_test.shape}  <- UP/DOWN (1/0)")
print(f"\n  Example:")
print(f"    Return:    {y_ret_test[0]:.2f}%")
print(f"    Direction: {y_dir_test[0]} ({'UP' if y_dir_test[0] == 1 else 'DOWN'})")
print(f"\n  Class distribution:")
print(f"    UP (1):   {np.sum(y_dir_test == 1)} ({np.mean(y_dir_test == 1)*100:.1f}%)")
print(f"    DOWN (0): {np.sum(y_dir_test == 0)} ({np.mean(y_dir_test == 0)*100:.1f}%)")


Test Sequence Creation on ABBOTINDIA:
  X shape:          (302, 60, 47)  <- (samples, window_size, features)
  y_return shape:   (302,)  <- 7-day return %
  y_direction shape: (302,)  <- UP/DOWN (1/0)

  Example:
    Return:    0.43%
    Direction: 1 (UP)

  Class distribution:
    UP (1):   142 (47.0%)
    DOWN (0): 160 (53.0%)


---
## 5. Train/Test Split & Data Preparation

In [14]:
# Split stocks into train and test sets
stock_symbols = list(stock_data.keys())
n_test_stocks = int(len(stock_symbols) * TEST_STOCK_RATIO)

np.random.seed(RANDOM_STATE)
shuffled_symbols = np.random.permutation(stock_symbols)

train_symbols = shuffled_symbols[:-n_test_stocks]
test_symbols = shuffled_symbols[-n_test_stocks:]

print(f"\nStock Split:")
print(f"  Training stocks: {len(train_symbols)} ({len(train_symbols)/len(stock_symbols)*100:.1f}%)")
print(f"  Test stocks:     {len(test_symbols)} ({len(test_symbols)/len(stock_symbols)*100:.1f}%)")


Stock Split:
  Training stocks: 79 (80.6%)
  Test stocks:     19 (19.4%)


In [15]:
# Create sequences for training stocks
print("\nCreating sequences for training stocks...")
X_train_list, y_return_train_list, y_dir_train_list = [], [], []

for symbol in tqdm(train_symbols, desc="Processing train stocks"):
    df = stock_data[symbol]
    X, y_ret, y_dir = create_sequences_with_returns(df, WINDOW_SIZE, HORIZON)
    X_train_list.append(X)
    y_return_train_list.append(y_ret)
    y_dir_train_list.append(y_dir)

X_train = np.concatenate(X_train_list, axis=0)
y_return_train = np.concatenate(y_return_train_list, axis=0)
y_direction_train = np.concatenate(y_dir_train_list, axis=0)

print(f"\nTraining set:")
print(f"  X_train: {X_train.shape}")
print(f"  y_return_train: {y_return_train.shape}")
print(f"  y_direction_train: {y_direction_train.shape}")
print(f"  UP/DOWN ratio: {np.mean(y_direction_train == 1)*100:.1f}% UP")


Creating sequences for training stocks...


Processing train stocks:   0%|          | 0/79 [00:00<?, ?it/s]


Training set:
  X_train: (23858, 60, 47)
  y_return_train: (23858,)
  y_direction_train: (23858,)
  UP/DOWN ratio: 50.1% UP


In [16]:
# Create sequences for test stocks
print("\nCreating sequences for test stocks...")
X_test_list, y_return_test_list, y_dir_test_list = [], [], []

for symbol in tqdm(test_symbols, desc="Processing test stocks"):
    df = stock_data[symbol]
    X, y_ret, y_dir = create_sequences_with_returns(df, WINDOW_SIZE, HORIZON)
    X_test_list.append(X)
    y_return_test_list.append(y_ret)
    y_dir_test_list.append(y_dir)

X_test = np.concatenate(X_test_list, axis=0)
y_return_test = np.concatenate(y_return_test_list, axis=0)
y_direction_test = np.concatenate(y_dir_test_list, axis=0)

print(f"\nTest set:")
print(f"  X_test: {X_test.shape}")
print(f"  y_return_test: {y_return_test.shape}")
print(f"  y_direction_test: {y_direction_test.shape}")
print(f"  UP/DOWN ratio: {np.mean(y_direction_test == 1)*100:.1f}% UP")


Creating sequences for test stocks...


Processing test stocks:   0%|          | 0/19 [00:00<?, ?it/s]


Test set:
  X_test: (5738, 60, 47)
  y_return_test: (5738,)
  y_direction_test: (5738,)
  UP/DOWN ratio: 51.7% UP


In [17]:
# Feature scaling (IMPORTANT: Don't scale targets - returns are already normalized)
print("\nScaling features...")

n_train_samples, window_size, n_features = X_train.shape
n_test_samples = X_test.shape[0]

X_train_2d = X_train.reshape(-1, n_features)
X_test_2d = X_test.reshape(-1, n_features)

scaler_X = StandardScaler()
X_train_scaled_2d = scaler_X.fit_transform(X_train_2d)
X_test_scaled_2d = scaler_X.transform(X_test_2d)

X_train_scaled = X_train_scaled_2d.reshape(n_train_samples, window_size, n_features)
X_test_scaled = X_test_scaled_2d.reshape(n_test_samples, window_size, n_features)

print(f"  X_train_scaled: {X_train_scaled.shape}")
print(f"  X_test_scaled:  {X_test_scaled.shape}")
print(f"\n  Targets NOT scaled (returns are already normalized)")


Scaling features...
  X_train_scaled: (23858, 60, 47)
  X_test_scaled:  (5738, 60, 47)

  Targets NOT scaled (returns are already normalized)


---
## 6. Helper Functions for Evaluation

In [18]:
def calculate_regression_metrics(y_true, y_pred):
    """Calculate metrics for return prediction (regression)."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
    
    return {
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'mape': mape
    }


def calculate_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate metrics for direction prediction (classification)."""
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    metrics = {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    }
    
    if y_pred_proba is not None:
        try:
            auc = roc_auc_score(y_true, y_pred_proba)
            metrics['auc'] = auc
        except:
            metrics['auc'] = 0.0
    
    return metrics


def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    """Calculate Sharpe ratio from returns."""
    if len(returns) == 0 or np.std(returns) == 0:
        return 0.0
    return (np.mean(returns) - risk_free_rate) / np.std(returns)


def simulate_trading(y_direction_true, y_direction_pred, y_return_true,
                    position_size=1000, transaction_cost=0.001):
    """
    Simulate trading with fair capital allocation for model comparison.

    Key Principle:
    - Each trade gets the SAME position size (₹1,000 by default)
    - Initial capital = position_size × number of UP predictions
    - This ensures models with different trade frequencies are fairly compared
    - Total return % is directly comparable across models

    Strategy:
    - If predict UP (1): BUY with position_size and hold for 7 days
    - If predict DOWN (0): Stay in cash (no trade)

    Args:
        position_size: Amount invested per trade (default ₹1,000)
        transaction_cost: Transaction cost as fraction (0.001 = 0.1%)

    Returns:
        Dictionary with trading performance metrics
    """
    # Count number of UP predictions (trades we'll make)
    n_trades = np.sum(y_direction_pred == 1)

    if n_trades == 0:
        return {
            'initial_capital': 0,
            'final_capital': 0,
            'total_return_pct': 0,
            'n_trades': 0,
            'win_rate': 0,
            'avg_return_per_trade': 0,
            'sharpe_ratio': 0,
            'trades': []
        }

    # Initial capital proportional to number of trades
    initial_capital = position_size * n_trades

    # Track all trades
    trades = []
    returns_list = []
    wins = 0
    total_pnl = 0

    for i in range(len(y_direction_pred)):
        pred_direction = y_direction_pred[i]
        actual_return_pct = y_return_true[i]

        # Only trade if we predict UP
        if pred_direction == 1:
            # Calculate net return after transaction costs
            # Cost = 0.1% on buy + 0.1% on sell = 0.2% total
            net_return_pct = actual_return_pct - (transaction_cost * 100 * 2)

            # P&L for this trade
            pnl = position_size * (net_return_pct / 100)
            total_pnl += pnl

            returns_list.append(net_return_pct)

            if net_return_pct > 0:
                wins += 1

            trades.append({
                'index': i,
                'predicted': pred_direction,
                'actual_direction': y_direction_true[i],
                'return_pct': actual_return_pct,
                'net_return': net_return_pct
            })

    # Final capital
    final_capital = initial_capital + total_pnl

    # Total return percentage
    total_return_pct = (total_pnl / initial_capital) * 100

    # Metrics
    win_rate = wins / n_trades
    avg_return_per_trade = np.mean(returns_list)
    sharpe_ratio = calculate_sharpe_ratio(returns_list)

    return {
        'initial_capital': initial_capital,
        'final_capital': final_capital,
        'total_return_pct': total_return_pct,
        'n_trades': n_trades,
        'win_rate': win_rate,
        'avg_return_per_trade': avg_return_per_trade,
        'sharpe_ratio': sharpe_ratio,
        'trades': trades
    }


def print_evaluation_summary(model_name, reg_metrics, clf_metrics, trading_metrics):
    """Print comprehensive evaluation summary."""
    print(f"\n{'='*70}")
    print(f"Results for {model_name}")
    print(f"{'='*70}")
    
    print(f"\n[REGRESSION] Return Prediction:")
    print(f"  MAE:   {reg_metrics['mae']:.3f}%")
    print(f"  RMSE:  {reg_metrics['rmse']:.3f}%")
    print(f"  R²:    {reg_metrics['r2']:.4f}")
    print(f"  MAPE:  {reg_metrics['mape']:.2f}%")
    
    print(f"\n[CLASSIFICATION] Direction Prediction:")
    print(f"  Accuracy:  {clf_metrics['accuracy']*100:.2f}%")
    print(f"  Precision: {clf_metrics['precision']*100:.2f}%")
    print(f"  Recall:    {clf_metrics['recall']*100:.2f}%")
    print(f"  F1-Score:  {clf_metrics['f1']*100:.2f}%")
    if 'auc' in clf_metrics:
        print(f"  ROC-AUC:   {clf_metrics['auc']:.4f}")
    
    print(f"\n[TRADING SIMULATION]:")
    print(f"  Initial Capital:   ₹{trading_metrics['initial_capital']:,.2f} ({trading_metrics['n_trades']} trades × ₹1,000)")
    print(f"  Final Capital:     ₹{trading_metrics['final_capital']:,.2f}")
    print(f"  Total Return:      {trading_metrics['total_return_pct']:.2f}%")
    print(f"  Number of Trades:  {trading_metrics['n_trades']}")
    print(f"  Win Rate:          {trading_metrics['win_rate']*100:.2f}%")
    print(f"  Avg Return/Trade:  {trading_metrics['avg_return_per_trade']:.3f}%")
    print(f"  Sharpe Ratio:      {trading_metrics['sharpe_ratio']:.4f}")
    
    print(f"{'='*70}\n")


print("Evaluation helper functions defined")

Evaluation helper functions defined


---
## 7. Baseline Models (with Advanced Features)

### 7.1 Linear Models with Class Weighting (Phase 3)

In [19]:
# Linear models with class_weight='balanced'
print("Running Linear Models with Class Weighting...\n")

with mlflow.start_run(run_name="Linear_ClassWeighted_47features"):
    mlflow.log_param("model_type", "Linear_ClassWeighted")
    mlflow.log_param("window_size", WINDOW_SIZE)
    mlflow.log_param("horizon", HORIZON)
    mlflow.log_param("num_features", len(feature_cols))
    mlflow.log_param("class_weight", "balanced")

    start_time = time.time()

    # Flatten inputs
    X_train_flat = X_train_scaled.reshape(len(X_train_scaled), -1)
    X_test_flat = X_test_scaled.reshape(len(X_test_scaled), -1)

    # Train return predictor (regression)
    print("Training Linear Regression for returns...")
    reg_model = LinearRegression()
    reg_model.fit(X_train_flat, y_return_train)
    y_return_pred = reg_model.predict(X_test_flat)

    # Train direction predictor (classification) with CLASS WEIGHTING
    print("Training Logistic Regression for direction (with class_weight='balanced')...")
    clf_model = LogisticRegression(
        max_iter=1000,
        random_state=RANDOM_STATE,
        class_weight='balanced'  # Phase 3: Class weighting
    )
    clf_model.fit(X_train_flat, y_direction_train)
    y_direction_pred_linear = clf_model.predict(X_test_flat)
    y_direction_pred_proba_linear = clf_model.predict_proba(X_test_flat)[:, 1]

    train_time = time.time() - start_time

    # Evaluate
    reg_metrics = calculate_regression_metrics(y_return_test, y_return_pred)
    clf_metrics = calculate_classification_metrics(y_direction_test, y_direction_pred_linear, y_direction_pred_proba_linear)
    trading_metrics = simulate_trading(y_direction_test, y_direction_pred_linear, y_return_test)

    # Log to MLflow
    for key, val in {**reg_metrics, **clf_metrics, **trading_metrics}.items():
        if key != 'trades':
            mlflow.log_metric(key, val)
    mlflow.log_metric("train_time_seconds", train_time)

    # Save for later ensemble
    linear_accuracy = clf_metrics['accuracy']
    linear_sharpe = trading_metrics['sharpe_ratio']

    print_evaluation_summary("Linear Models (Class Weighted)", reg_metrics, clf_metrics, trading_metrics)
    print(f"Training time: {train_time:.2f}s")

print("Linear models complete")

Running Linear Models with Class Weighting...

Training Linear Regression for returns...
Training Logistic Regression for direction (with class_weight='balanced')...

Results for Linear Models (Class Weighted)

[REGRESSION] Return Prediction:
  MAE:   3.346%
  RMSE:  4.376%
  R²:    0.1093
  MAPE:  11463036.81%

[CLASSIFICATION] Direction Prediction:
  Accuracy:  63.47%
  Precision: 65.99%
  Recall:    60.52%
  F1-Score:  63.14%
  ROC-AUC:   0.6832

[TRADING SIMULATION]:
  Initial Capital:   ₹2,720,000.00 (2720 trades × ₹1,000)
  Final Capital:     ₹2,758,840.67
  Total Return:      1.43%
  Number of Trades:  2720
  Win Rate:          63.93%
  Avg Return/Trade:  1.428%
  Sharpe Ratio:      0.3227

Training time: 213.82s
Linear models complete


### 7.2 XGBoost Classifier (Phase 2)

In [20]:
# XGBoost for direction prediction
print("\nRunning XGBoost Classifier...\n")

with mlflow.start_run(run_name="XGBoost_47features"):
    mlflow.log_param("model_type", "XGBoost")
    mlflow.log_param("window_size", WINDOW_SIZE)
    mlflow.log_param("horizon", HORIZON)
    mlflow.log_param("num_features", len(feature_cols))
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("n_estimators", 100)

    start_time = time.time()

    # Flatten inputs
    X_train_flat = X_train_scaled.reshape(len(X_train_scaled), -1)
    X_test_flat = X_test_scaled.reshape(len(X_test_scaled), -1)

    # Train XGBoost
    print("Training XGBoost for direction...")
    xgb_clf = XGBClassifier(
        max_depth=6,
        learning_rate=0.1,
        n_estimators=100,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        eval_metric='logloss'
    )
    xgb_clf.fit(X_train_flat, y_direction_train)
    y_direction_pred_xgb = xgb_clf.predict(X_test_flat)
    y_direction_pred_proba_xgb = xgb_clf.predict_proba(X_test_flat)[:, 1]

    train_time = time.time() - start_time

    # Evaluate
    clf_metrics = calculate_classification_metrics(y_direction_test, y_direction_pred_xgb, y_direction_pred_proba_xgb)
    trading_metrics = simulate_trading(y_direction_test, y_direction_pred_xgb, y_return_test)

    # Log to MLflow
    for key, val in {**clf_metrics, **trading_metrics}.items():
        if key != 'trades':
            mlflow.log_metric(key, val)
    mlflow.log_metric("train_time_seconds", train_time)

    # Save for ensemble
    xgb_accuracy = clf_metrics['accuracy']
    xgb_sharpe = trading_metrics['sharpe_ratio']

    # Dummy reg_metrics for print function
    reg_metrics = {'mae': 0, 'rmse': 0, 'r2': 0, 'mape': 0}
    print_evaluation_summary("XGBoost", reg_metrics, clf_metrics, trading_metrics)
    print(f"Training time: {train_time:.2f}s")

print("XGBoost complete")


Running XGBoost Classifier...

Training XGBoost for direction...

Results for XGBoost

[REGRESSION] Return Prediction:
  MAE:   0.000%
  RMSE:  0.000%
  R²:    0.0000
  MAPE:  0.00%

[CLASSIFICATION] Direction Prediction:
  Accuracy:  70.08%
  Precision: 73.10%
  Recall:    66.62%
  F1-Score:  69.71%
  ROC-AUC:   0.7662

[TRADING SIMULATION]:
  Initial Capital:   ₹2,703,000.00 (2703 trades × ₹1,000)
  Final Capital:     ₹2,759,009.45
  Total Return:      2.07%
  Number of Trades:  2703
  Win Rate:          70.81%
  Avg Return/Trade:  2.072%
  Sharpe Ratio:      0.4992

Training time: 109.94s
XGBoost complete


### 7.3 Ensemble: Linear + XGBoost (70-30 weighted)

In [21]:
# Weighted ensemble: Linear (70%) + XGBoost (30%)
print("\nRunning Ensemble (Linear + XGBoost)...\n")

with mlflow.start_run(run_name="Ensemble_Linear70_XGB30_47features"):
    mlflow.log_param("model_type", "Ensemble_Linear_XGBoost")
    mlflow.log_param("window_size", WINDOW_SIZE)
    mlflow.log_param("horizon", HORIZON)
    mlflow.log_param("num_features", len(feature_cols))
    mlflow.log_param("linear_weight", 0.7)
    mlflow.log_param("xgboost_weight", 0.3)

    # Weighted ensemble probabilities
    alpha = 0.7  # Linear weight
    beta = 0.3   # XGBoost weight
    y_ensemble_proba = alpha * y_direction_pred_proba_linear + beta * y_direction_pred_proba_xgb
    y_direction_pred_ensemble = (y_ensemble_proba > 0.5).astype(int)

    # Evaluate
    clf_metrics = calculate_classification_metrics(y_direction_test, y_direction_pred_ensemble, y_ensemble_proba)
    trading_metrics = simulate_trading(y_direction_test, y_direction_pred_ensemble, y_return_test)

    # Log to MLflow
    for key, val in {**clf_metrics, **trading_metrics}.items():
        if key != 'trades':
            mlflow.log_metric(key, val)

    # Dummy reg_metrics
    reg_metrics = {'mae': 0, 'rmse': 0, 'r2': 0, 'mape': 0}
    print_evaluation_summary("Ensemble (Linear 70% + XGBoost 30%)", reg_metrics, clf_metrics, trading_metrics)

    ensemble_accuracy = clf_metrics['accuracy']
    ensemble_sharpe = trading_metrics['sharpe_ratio']

print("\n" + "="*80)
print("PHASE 1-3 RESULTS SUMMARY")
print("="*80)
print(f"Baseline (04a - 32 features, no class weight):  58.81% accuracy, 0.2095 Sharpe")
print(f"Phase 1 (47 features, class weight):            {linear_accuracy*100:.2f}% accuracy, {linear_sharpe:.4f} Sharpe")
print(f"Phase 2 (XGBoost only):                          {xgb_accuracy*100:.2f}% accuracy, {xgb_sharpe:.4f} Sharpe")
print(f"Phase 3 (Ensemble):                              {ensemble_accuracy*100:.2f}% accuracy, {ensemble_sharpe:.4f} Sharpe")

print("\n✅ GOAL STATUS:")
if ensemble_accuracy > 0.60:
    print(f"   🎯 SUCCESS! Accuracy {ensemble_accuracy*100:.2f}% exceeds 60% target")
elif linear_accuracy > 0.60:
    print(f"   🎯 SUCCESS! Linear model {linear_accuracy*100:.2f}% exceeds 60% target")
else:
    print(f"   ⚠️  Not yet at 60% target. Consider Phase 4 (per-stock models)")

print("="*80)


Running Ensemble (Linear + XGBoost)...


Results for Ensemble (Linear 70% + XGBoost 30%)

[REGRESSION] Return Prediction:
  MAE:   0.000%
  RMSE:  0.000%
  R²:    0.0000
  MAPE:  0.00%

[CLASSIFICATION] Direction Prediction:
  Accuracy:  66.52%
  Precision: 69.16%
  Recall:    63.59%
  F1-Score:  66.26%
  ROC-AUC:   0.7248

[TRADING SIMULATION]:
  Initial Capital:   ₹2,727,000.00 (2727 trades × ₹1,000)
  Final Capital:     ₹2,775,286.98
  Total Return:      1.77%
  Number of Trades:  2727
  Win Rate:          67.25%
  Avg Return/Trade:  1.771%
  Sharpe Ratio:      0.4161


PHASE 1-3 RESULTS SUMMARY
Baseline (04a - 32 features, no class weight):  58.81% accuracy, 0.2095 Sharpe
Phase 1 (47 features, class weight):            63.47% accuracy, 0.3227 Sharpe
Phase 2 (XGBoost only):                          70.08% accuracy, 0.4992 Sharpe
Phase 3 (Ensemble):                              66.52% accuracy, 0.4161 Sharpe

✅ GOAL STATUS:
   🎯 SUCCESS! Accuracy 66.52% exceeds 60% target


---
## 8. Final Comparison with Baseline

In [22]:
# Query all runs from experiment
experiment = mlflow.get_experiment_by_name("stock-return-prediction-multihorizon")
runs_df = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.sharpe_ratio DESC"]
)

# Filter to show only our new runs + baseline
recent_runs = runs_df[runs_df['tags.mlflow.runName'].isin([
    'Linear_Models',  # Baseline from 04a
    'Linear_ClassWeighted_47features',
    'XGBoost_47features',
    'Ensemble_Linear70_XGB30_47features'
])]

print(f"\nComparison: Baseline vs New Approach")
print("="*100)

# Select columns
summary_cols = [
    'tags.mlflow.runName',
    'metrics.accuracy',
    'metrics.f1',
    'metrics.sharpe_ratio',
    'metrics.total_return_pct',
    'metrics.win_rate',
    'metrics.train_time_seconds'
]

summary_df = recent_runs[summary_cols].copy()
summary_df.columns = ['Model', 'Accuracy', 'F1', 'Sharpe', 'Total Return %', 'Win Rate', 'Train Time (s)']
summary_df = summary_df.sort_values('Sharpe', ascending=False)

print(summary_df.to_string(index=False))
print("="*100)

# Calculate improvements
baseline_row = summary_df[summary_df['Model'] == 'Linear_Models']
best_row = summary_df.iloc[0]

if len(baseline_row) > 0:
    baseline_acc = baseline_row['Accuracy'].values[0]
    baseline_sharpe = baseline_row['Sharpe'].values[0]
    best_acc = best_row['Accuracy']
    best_sharpe = best_row['Sharpe']

    acc_improvement = (best_acc - baseline_acc) * 100
    sharpe_improvement = ((best_sharpe - baseline_sharpe) / baseline_sharpe) * 100

    print(f"\n📊 IMPROVEMENT SUMMARY:")
    print(f"   Best Model: {best_row['Model']}")
    print(f"   Accuracy: {baseline_acc*100:.2f}% → {best_acc*100:.2f}% (+{acc_improvement:.2f}%)")
    print(f"   Sharpe: {baseline_sharpe:.4f} → {best_sharpe:.4f} (+{sharpe_improvement:.1f}%)")

    if best_acc > 0.60:
        print(f"\n   🎉 SUCCESS! Exceeded 60% accuracy target!")
    else:
        gap = (0.60 - best_acc) * 100
        print(f"\n   ⚠️  Still {gap:.2f}% away from 60% target")
        print(f"   💡 Next step: Implement Phase 4 (per-stock models)")


Comparison: Baseline vs New Approach
                             Model  Accuracy       F1   Sharpe  Total Return %  Win Rate  Train Time (s)
                XGBoost_47features  0.700767 0.697125 0.499166        2.072122  0.708102      109.938113
Ensemble_Linear70_XGB30_47features  0.665214 0.662568 0.416094        1.770700  0.672534             NaN
   Linear_ClassWeighted_47features  0.634716 0.631375 0.322729        1.427966  0.639338      213.824207
                     Linear_Models  0.588132 0.608020 0.209492        0.942082  0.585561      166.927659

📊 IMPROVEMENT SUMMARY:
   Best Model: XGBoost_47features
   Accuracy: 58.81% → 70.08% (+11.26%)
   Sharpe: 0.2095 → 0.4992 (+138.3%)

   🎉 SUCCESS! Exceeded 60% accuracy target!


---
## 9. Save Best Model for Production Use

In [23]:
# Save the best model (XGBoost) and scaler for production
import pickle

MODEL_SAVE_PATH = MODELS_DIR / "xgboost_stock_direction_predictor.pkl"
SCALER_SAVE_PATH = MODELS_DIR / "feature_scaler.pkl"

# Save XGBoost model
with open(MODEL_SAVE_PATH, 'wb') as f:
    pickle.dump(xgb_clf, f)

# Save scaler
with open(SCALER_SAVE_PATH, 'wb') as f:
    pickle.dump(scaler_X, f)

print(f"✅ Model saved to: {MODEL_SAVE_PATH}")
print(f"✅ Scaler saved to: {SCALER_SAVE_PATH}")
print(f"\nModel info:")
print(f"  - Type: XGBoost Classifier")
print(f"  - Accuracy: 70.08%")
print(f"  - Precision: 73.10%")
print(f"  - Sharpe Ratio: 0.4992")
print(f"  - Features: 47 (60-day window)")
print(f"  - Horizon: 7-day prediction")
print(f"\n💡 Use these files in notebook 10 for production filtering")

✅ Model saved to: models/xgboost_stock_direction_predictor.pkl
✅ Scaler saved to: models/feature_scaler.pkl

Model info:
  - Type: XGBoost Classifier
  - Accuracy: 70.08%
  - Precision: 73.10%
  - Sharpe Ratio: 0.4992
  - Features: 47 (60-day window)
  - Horizon: 7-day prediction

💡 Use these files in notebook 10 for production filtering


In [24]:
# Test loading the saved model
with open(MODEL_SAVE_PATH, 'rb') as f:
    loaded_model = pickle.load(f)

with open(SCALER_SAVE_PATH, 'rb') as f:
    loaded_scaler = pickle.load(f)

# Quick test on one sample
test_sample = X_test_flat[:1]
pred = loaded_model.predict(test_sample)
pred_proba = loaded_model.predict_proba(test_sample)

print("✅ Model loading test successful!")
print(f"   Sample prediction: {pred[0]} ({'UP' if pred[0] == 1 else 'DOWN'})")
print(f"   Probabilities: [DOWN: {pred_proba[0][0]:.2%}, UP: {pred_proba[0][1]:.2%}]")

✅ Model loading test successful!
   Sample prediction: 1 (UP)
   Probabilities: [DOWN: 34.44%, UP: 65.56%]


---
## Summary

### What We Built
1. **Advanced Feature Engineering**: Added 15 new technical features (47 total)
2. **Best Model**: XGBoost classifier with 70.08% accuracy (+11.26% over baseline)
3. **Saved for Production**: Model and scaler ready for notebook 10

### Performance Metrics
- **Accuracy**: 70.08% (target was >60%)
- **Precision**: 73.10% (when predicts UP, correct 73% of time)
- **Win Rate**: 70.81% in trading simulation
- **Sharpe Ratio**: 0.4992 (+138% over baseline)

### Next Steps
1. Open **notebook 10** ([10_production_ml_filter.ipynb](10_production_ml_filter.ipynb))
2. Provide LLM-recommended stock symbols as input
3. Get ML-filtered BUY recommendations

### Production Usage Example
```python
# In notebook 10
llm_stocks = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']
filtered = filter_llm_recommendations(llm_stocks, model, scaler)
buy_list = filtered[filtered['recommendation'] == 'BUY']
```